## TODO
make it more robust and generic. some links like "http://www.lemans.frjavascript:window.document.form1.submit();" are not passed
collect root url for detected external links

In [ ]:
import urllib3
import requests
import certifi

from bs4 import BeautifulSoup
from bs4.element import Comment

import pickle
import re

#import gensim 

In [ ]:
def save_pickle(filename, **kwargs):
    for key, value in kwargs.items():
        with open(filename+'-'+key+'.pickle', 'wb') as handle:
            pickle.dump(value, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def load_pickle(filename, links, ext_links, passed_links):
    with open(filename+'-links.pickle', 'rb') as handle:
        links = pickle.load(handle)            
    with open(filename+'-ext_links.pickle', 'rb') as handle:
        ext_links = pickle.load(handle)
    with open(filename+'-passed_links.pickle', 'rb') as handle:
        passed_links = pickle.load(handle)

    return links, ext_links, passed_links

In [ ]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

In [ ]:
def get_page_content(soup):
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

In [ ]:
def add_new_link(soup,links,url):
    #print('add new link', url)
    if url not in links:
        links[url] = False
        #print('added')
    #else:
        #print('passed')
    return links

In [ ]:
def get_page_links(soup, links, ext_links, passed_links, root):
    for l in soup.find_all('a', href=True):
        lURL =  l['href'].strip()
        #print('processing link:',lURL,':',len(lURL))
        if lURL == '/' \
        or lURL == '' \
        or lURL[:4] == 'tel:' \
        or lURL[:7] == 'mailto:' \
        or lURL[:11] == 'javascript:' \
        or lURL[-4:] == '.pdf' \
        or lURL == '#': 
            passed_links[lURL]=False
        elif (root not in lURL and lURL[:4] == 'http'):
            ext_links[lURL]=False
        elif root in lURL:
            links = add_new_link(soup,links,lURL)
        else:
            if lURL[0] == '.':
                lURL = lURL[1:]
            links = add_new_link(soup,links,root+lURL)
    return links, ext_links, passed_links

In [ ]:
def get_unread_page(links):
    lKey = [key for key, value in links.items() if value == False]
    if len(lKey)>0:
        #print('next url ',lKey[0])
        return lKey[0]
    else:
        print('no more url')
    return None

In [ ]:
#init
# get url to scan
root = 'http://www.lemans.fr'
#root = 'https://www.neuralia.co'

http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())
links = {}
ext_links = {}
passed_links = {}
mining = True
links[root] = False
chars_to_remove = ['.', '/', ':']
rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
filename = re.sub(rx, '', root)
links, ext_links, passed_links = load_pickle(filename, links, ext_links, passed_links)

In [ ]:
# iterate
N = 0
links, ext_links, passed_links = load_pickle(filename, links, ext_links, passed_links)

while mining is True:
    url = get_unread_page(links)
    print(f'main processing {url} | items in dict: {len(links)} | items to process: {sum(value == False for value in links.values())} ')
    if url == None:
        mining = False
        save_pickle(filename, links=links, ext_links=ext_links, passed_links=passed_links)
    else:
        response = http.request('GET', url)
        soup = BeautifulSoup(response.data,"html5lib")
        print('souped')
        links, ext_links, passed_links = get_page_links(soup, links, ext_links, passed_links, root)
        links[url] = get_page_content(soup)
        print(links[url][:50])
    
    if(not N % 100):
        print(f'pickling')
        save_pickle(filename, links=links, ext_links=ext_links, passed_links=passed_links)
    N +=1

In [ ]:
save_pickle(filename, links=links, ext_links=ext_links, passed_links=passed_links)

In [ ]:
for key,value in passed_links.items():
    print(key)
    #print(value)

In [ ]:
for key,value in ext_links.items():
    print(key)
    #print(value)

In [ ]:
for key,value in links.items():
    print(key)
    print(value == False)

In [ ]:
links['http://www.lemans.fr/nos-autres-sites/consulter-le-site-de-la-cite-du-cirque/']

In [ ]:
save_pickle(filename, links=links, ext_links=ext_links, passed_links=passed_links)

In [ ]:
a= ["http://www.lemans.frjavascript:linkTo_UnCryptMailto('nbjmup+kfbo.nbsd\/epsnfbvAmbqptuf\/ofu');",
   "http://www.lemans.frjavascript:window.document.form1.submit();",
   "http://www.lemans.frjavascript:void(0)",
   "http://www.lemans.frjavascript:linkTo_UnCryptMailto('nbjmup+stnboefmb\/tfdsfubsjbuAxbobepp\/gs');",
    "http://www.lemans.frjavascript:linkTo_UnCryptMailto('nbjmup+tbsuif.nbzfoofAibcjubu.ivnbojtnf\/psh');",
   "http://www.lemans.frjavascript:linkTo_UnCryptMailto('nbjmup+eeu.bobiAtbsuif\/hpvw\/gs');"]
a=["http://www.lemans.frindex.do","http://www.lemans.frtheme_oublimdp_afficher.do","http://www.lemans.frmentions_legales.do","http://www.lemans.frentete","http://www.lemans.fr/dynamique/bienvenue/les-ambassadeurs/les-portraits/#firstMenu"]
a=["http://www.lemans.frhtpp://www.coprugbylemans.com"]
a=["http://www.lemans.frhtpp://musicalemans.fr/"]

In [ ]:
b=["htpp://www.coprugbylemans.com","htpp://musicalemans.fr/"]

In [ ]:
for a1 in a:
    print(a1)
    try:
        del links[a1]
    except:
        print(a1,' not found')
        
for b1 in b:
    ext_links[b1]=False
    
save_pickle(filename, links=links, ext_links=ext_links, passed_links=passed_links)

In [ ]:
#get domain of external links

In [ ]:
#ride links contents